In [71]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

In [72]:
from sklearn.datasets import load_breast_cancer
dataset = load_breast_cancer()
X = dataset.data
y = dataset.target

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state =3)

In [4]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
knn.score(X_test,y_test)

0.916083916083916

let's using bagging over our KNN classifier and see if our score improves:

In [5]:
bag_knn = BaggingClassifier(KNeighborsClassifier(n_neighbors=5),
                  n_estimators = 10,
                  max_samples=0.5,
                 bootstrap = True,random_state = 3,
                 oob_score = True)

In [7]:
#Let's check the out of bag score 
bag_knn.oob_score

True

In [8]:
bag_knn.fit(X_train,y_train)
bag_knn.score(X_test,y_test)

0.9370629370629371

Great! our score sginificantly improves with use of bagging.

let's not use bootstrap and see the model accuracy! Remember this is "Pasting"

In [9]:
pasting_knn = BaggingClassifier(KNeighborsClassifier(n_neighbors=5),
                 n_estimators=10, max_samples=0.5,
                 bootstrap=False, random_state=3)

In [10]:
pasting_knn.fit(X_train, y_train)
pasting_knn.score(X_test,y_test)

0.9300699300699301

In [3]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score
from sklearn.tree import export_graphviz


In [4]:
data = pd.read_csv(r"C:\Users\Admin\Downloads\code form iNeuron\EnsembleLearning_And_RandomForest\winequality_red.csv")
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [5]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


The data set consists following Input variables :
1 - fixed acidity  2 - volatile acidity  3 - citric acid  4 - residual sugar  5 - chlorides  6 - free sulfur dioxide

7 - total sulfur dioxide  8 - density  9 - pH   10 - sulphates   11 - alcohol

and the Output variable gives the quality of th wine based on the input variables: 

12 - quality (score between 0 and 10)

In [6]:
data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [7]:
X = data.drop(columns = 'quality')
y = data['quality']

In [8]:
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size = 0.30, random_state = 355 )

In [9]:
#let's first visualize the tree on the data without doing any pre processing
clf = DecisionTreeClassifier(min_samples_split = 2)
clf.fit(x_train, y_train)    

DecisionTreeClassifier()

In [10]:
# accuracy of our classification tree
clf.score(x_test, y_test)

0.6354166666666666

In [11]:
#let's first visualize the tree on the data without doing any pre processing
clf2 = DecisionTreeClassifier(criterion = 'entropy', max_depth = 24, min_samples_leaf = 1)
clf.fit(x_train, y_train)

DecisionTreeClassifier()

In [12]:
clf.score(x_test, y_test)

0.6270833333333333

In [13]:
rand_clf = RandomForestClassifier(random_state = 6)

Random state, if given none then score will vary everytime you run the RandomForestClassifier. If we asssign a value to it, then result will remain constant.

In [14]:
rand_clf.fit(x_train,y_train)

RandomForestClassifier(random_state=6)

In [15]:
rand_clf.score(x_test,y_test)

0.6708333333333333

We can see that two individual decision trees have both less score than a single random forest classifier. 

So, using random forest classifier has increased the predicitive power of our model. 

Great, let's do some hyperparameter tuning and see if we can increase our accuracy more.

Random forest hyperparameters are a combination of best hyperparameters of both decision tree and Bagging classifier.

* Hyperparameters of Decision tree: 

class_weight=None, criterion='entropy', max_depth=24,max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best'
                    
* Hyperparameters of Bagging classifier:

base_estimator=None, bootstrap=True, bootstrap_features=False,
                  max_features=1.0, max_samples=1.0, n_estimators=10,
                  n_jobs=None, oob_score=False, random_state=None, verbose=0,
                  warm_start=False
                  
* Hyperparameters of Random forest classifier:

bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False

Let's now try to tune some hyperparameters using the GridSearchCV algorithm.
We have studied about CrossValidation in upcoming lecture. 

GridSearchCV is a method used to tune our hyperparameters. We can pass different values of hyperparameters as parameters for grid search.
It does a exhaustive generation of combination of different parameters passed.
Using cross validation score, Grid Search returns the combination of hyperparameters for which the model is performing the best. 

Note that it is common that a small subset of those parameters can have a large impact on the predictive or computation performance of the model while others can be left to their default values. 

In [16]:
# we are tuning three hyperparameters right now, we are passing the different values for both parameters
grid_param = {
    "n_estimators" : [90, 100, 115, 130],
    'criterion' : ['gini', 'entropy'],
    'max_depth' : range(2,20,1),
    'min_samples_leaf' : range(1,10,1),
    'min_samples_split' :range(2,10,1),
    'max_features' : ['auto', 'log2']
}    
             

In [17]:
grid_search = GridSearchCV(estimator = rand_clf,param_grid = grid_param, cv=5,n_jobs = -1, verbose = 3 )

In [ ]:
grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 20736 candidates, totalling 103680 fits


In [2]:
#let's see the best parameters as per our grid search
grid_search.best_params_

NameError: name 'grid_search' is not defined

We will pass these parameters into our random forest classifier.

In [19]:
rand_clf = RandomForestClassifier(criterion = 'entropy',
                                 max_depth =12,
                                 max_features = 'log2',
                                 min_samples_leaf =1,
                                 min_samples_split= 5,
                                 n_estimators = 90, random_state= 6)

In [20]:
rand_clf.fit(x_train,y_train)

RandomForestClassifier(criterion='entropy', max_depth=12, max_features='log2',
                       min_samples_split=5, n_estimators=90, random_state=6)

In [21]:
rand_clf.score(x_test,y_test)

0.6604166666666667

Great! Our accuracy has increased by 2% after using the best parameters for GridsearchCV.

Let's do some more tweak in the hyper parameters and try gridSearch on it.

In [24]:
# we are tuning three hyperparameters right now, we are passing the different values for both parameters
grid_param = {
    'n_estimators' : [90,100,115],
    'criterion' : ['gini', 'entropy'],
    'min_samples_leaf' : [1,2,3,4,5],
    'min_samples_split' : [4,5,6,7,8],
    'max_features' : ['auto',' log2']
}

In [25]:
grid_search = GridSearchCV(estimator=rand_clf,param_grid=grid_param,cv=5,n_jobs=-1,verbose=3)

In [26]:
grid_search.fit(x_train,y_train)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


C:\Users\Admin\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.65683056 0.6559417  0.65595372 0.66220372 0.66040999 0.66310058
 0.65595372 0.65685058 0.6586443  0.65325112 0.65234225 0.65324712
 0.66489029 0.66578716 0.66132687 0.66131086 0.66131486 0.65863629
 0.67202514 0.67114029 0.66935058 0.66669202 0.67115231 0.66133088
 0.65862028 0.66220372 0.66130285 0.65505685 0.66131486 0.66132287
 0.65683857 0.65951714 0.65862028 0.65683857 0.65951714 0.65862028
 0.65683857 0.65951714 0.65862028 0.65595772 0.65596573 0.65684257
 0.66130285 0.66398142 0.66309657 0.64970772 0.65328716 0.65060458
 0.64970772 0.65328716 0.65060458 0.64970772 0.65328716 0.65060458
 0.64970772 0.65328716 0.65060458 0.64970772 0.65328716 0.65060458
 0.65595372 0.65059657 0.65505685 0.65595372 0.65059657 0.65505685
 0.65595372 0.65059657 0.65505685 0.65595372 0.65059657 0.65505685
 0.65595372 0.65059657 0.65505685        nan        n

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(criterion='entropy', max_depth=12,
                                              max_features='log2',
                                              min_samples_split=5,
                                              n_estimators=90, random_state=6),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', ' log2'],
                         'min_samples_leaf': [1, 2, 3, 4, 5],
                         'min_samples_split': [4, 5, 6, 7, 8],
                         'n_estimators': [90, 100, 115]},
             verbose=3)

In [27]:
#let's see the best parameters as per our grid search
grid_search.best_params_

{'criterion': 'entropy',
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 7,
 'n_estimators': 90}

In [28]:
rand_clf = RandomForestClassifier(criterion = 'entropy',
                      max_features = 'auto',
                      min_samples_leaf = 1,
                      min_samples_split = 4,
                      n_estimators = 115,
                      random_state = 6)

In [29]:
rand_clf.fit(x_train,y_train)

RandomForestClassifier(criterion='entropy', min_samples_split=4,
                       n_estimators=115, random_state=6)

In [30]:
rand_clf.score(x_test,y_test)

0.6729166666666667

Our accuracy has improved and score is better than the last grid search. So, we can say that giving all the hyperparameters in the gridSearch 
doesn't gurantee the best result. We have to do hit and trial with parameters to get the perfect score.

You are welcome to try tweaking the parameters more and try an improve the accuracy more.



In [36]:
# let's save the model
import pickle
with open(r"C:\Users\Admin\Downloads\code form iNeuron\EnsembleLearning_And_RandomForest"+ '/modelForPrediction.sav', 'wb') as f:
    pickle.dump(rand_clf,f)

# Stacking (Stacked Generalization)

Stacking is a type of ensemble technique which combines the predictions of two or more models, also called base models, and use the combination as the input for a new model (meta-model) i.e. a new model is trained on the predictions of the base models. 

Although, there is no libraries available in Sklearn for stacking, it can still be implemented.

Let's understand more about stacking with python implementation:


In [37]:
# Let's use the same dataset used above for Random forest classification and try to improve the accuracy more using stacking.
data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [73]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split    

In [40]:
data = pd.read_csv(r"C:\Users\Admin\Downloads\code form iNeuron\EnsembleLearning_And_RandomForest\diabetes.csv")
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [41]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [43]:
X = data.drop(columns = 'Outcome')
y = data['Outcome']

In [44]:
# let's divide our dataset into training set and hold out set by 50%
train,val_train,test,val_test = train_test_split(X,y,test_size = 0.5, random_state  =355)

In [45]:
# let's split the training set again into training and test dataset 
x_train,x_test,y_train,y_test  = train_test_split(train,test,test_size = 0.2,random_state= 355)

In [46]:
knn = KNeighborsClassifier()
knn.fit(x_train,y_train)

KNeighborsClassifier()

In [47]:
knn.score(x_test,y_test)

0.7402597402597403

In [50]:
# rand_clf = RandomForestClassifier()

# rand_clf.fit(x_train,y_train)


svm = SVC()
svm.fit(x_train,y_train)

SVC()

In [51]:
#rand_clf.score(x_test,y_test)
svm.score(x_test,y_test)

0.7402597402597403

Let's get the predictions of all the base models on the validation set val_train.

In [53]:
predict_val1 = knn.predict(val_train)
predict_val2 = svm.predict(val_train)
#predict_val2 = rand_clf.predict(val_train)

Let's stack the prediciton values for validation set together as "predict_val"

In [62]:
predict_val = np.column_stack((predict_val1,predict_val2))
predict_val

array([[0, 0],
       [0, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [1, 0],
       [0, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [1, 1],
       [1, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [1, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [1, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [1, 0],
       [1, 1],
       [1, 1],
       [1,

Let's get the prediction for all the base models on the test set  x_test

In [63]:
predict_test1 = knn.predict(x_test)
predict_test2 = svm.predict(x_test)
#predict_val2  = rand_clf(x_test)

Let's stack the prediciton values for validation set together as "predict_test"

In [64]:
predict_test = np.column_stack((predict_test1,predict_test2))
predict_test

array([[1, 0],
       [0, 0],
       [1, 1],
       [1, 0],
       [0, 0],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [1, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 1],
       [1, 1],
       [1, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [1, 1],
       [0, 0],
       [1, 1],
       [1, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [1,

Let's use the Use the stacked data "predict_val" and val_test as the input feature for meta_model i.e. randomforest classifier

In [65]:
# svm = SVC()
#svm.fit(predict_val,val_test)

In [66]:
#svm.score(predict_test,y_test)

In [74]:
rand_clf = RandomForestClassifier()
rand_clf.fit(predict_val,val_test)

RandomForestClassifier()

In [75]:
rand_clf.score(predict_test,y_test)

0.7402597402597403

In [80]:
# we are tuning three hyperparameters right now, we are passing the different values for both parameters
grid_param = {
    'n_estimators' : [90,100,115],
    'criterion' : ['gini' , 'entropy'],
    'min_samples_leaf': [1,2,3,4,5],
    'min_samples_split': [4,5,6,7,8],
    'max_features': ['auto', 'log2']
}

In [81]:
grid_search = GridSearchCV(estimator=rand_clf, param_grid = grid_param, cv = 5, n_jobs = -1, verbose = 3)
grid_search

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'log2'],
                         'min_samples_leaf': [1, 2, 3, 4, 5],
                         'min_samples_split': [4, 5, 6, 7, 8],
                         'n_estimators': [90, 100, 115]},
             verbose=3)

In [82]:
grid_search.fit(predict_val,val_test)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'log2'],
                         'min_samples_leaf': [1, 2, 3, 4, 5],
                         'min_samples_split': [4, 5, 6, 7, 8],
                         'n_estimators': [90, 100, 115]},
             verbose=3)

In [84]:
grid_search.best_params_

{'criterion': 'gini',
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 4,
 'n_estimators': 90}

In [85]:
rand_clf = RandomForestClassifier(criterion = 'gini', max_features = 'auto', min_samples_leaf = 1,min_samples_split = 4,n_estimators = 90)

In [86]:
rand_clf.fit(predict_val,val_test)

RandomForestClassifier(min_samples_split=4, n_estimators=90)

In [87]:
rand_clf.score(predict_test,y_test)

0.7402597402597403